In [45]:
import sys
sys.path.append('..')
import sql
import pandas as pd
from functools import reduce
from tabulate import tabulate
import datetime
import matplotlib.pyplot as plt
import numpy as np

In [46]:
d= {'Is there a documentation source?': 'is_doc','is the fix mentioned?':'is_fix', 'is there unrelated change mentioned?': 'is_uc', 'is breaking change mentioned?':'is_br' , 'main source' :'doc_source', 'How is the fix mentioned? (final codes); separated by semicolon':'fix_pattern',
'how the breaking change is mentioned? (final codes); separated by semicolon':'breaking_pattern', 'main source':'source'}
aniqa = pd.read_csv('process-frozen-aniqa.csv')
aniqa = aniqa.rename(columns = d)
nasif = pd.read_csv('process-frozen-nasif.csv')
nasif = nasif.rename(columns = d)

In [47]:
aniqa['is_doc'] = aniqa['is_doc'].fillna('N')
nasif['is_doc'] = nasif['is_doc'].fillna('N')
aniqa['is_doc'] = aniqa['is_doc'].str.lower()
nasif['is_doc'] = nasif['is_doc'].str.lower()
a = np.where(aniqa['is_doc'] != nasif['is_doc'])
d = set(list(a[0]))
aniqa['is_fix'] = aniqa['is_fix'].fillna('N')
nasif['is_fix'] = nasif['is_fix'].fillna('N')
aniqa['is_fix'] = aniqa['is_fix'].str.lower()
nasif['is_fix'] = nasif['is_fix'].str.lower()
a = np.where(aniqa['is_fix'] != nasif['is_fix'])
f = set(list(a[0]))
aniqa['is_uc'] = aniqa['is_uc'].fillna('N')
nasif['is_uc'] = nasif['is_uc'].fillna('N')
aniqa['is_uc'] = aniqa['is_uc'].str.lower()
nasif['is_uc'] = nasif['is_uc'].str.lower()
a = np.where(aniqa['is_uc'] != nasif['is_uc'])
u = set(list(a[0]))
aniqa['is_br'] = aniqa['is_br'].fillna('N')
nasif['is_br'] = nasif['is_br'].fillna('N')
aniqa['is_br'] = aniqa['is_br'].str.lower()
nasif['is_br'] = nasif['is_br'].str.lower()
a = np.where(aniqa['is_br'] != nasif['is_br'])
b = set(list(a[0]))
s=d.union(f).union(u).union(b)
len(s)

137

In [48]:
nasif['fix_pattern'] = nasif['fix_pattern'].fillna('')
aniqa['fix_pattern'] = aniqa['fix_pattern'].fillna('')
nasif['breaking_pattern'] = nasif['breaking_pattern'].fillna('')
aniqa['breaking_pattern'] = aniqa['breaking_pattern'].fillna('')
nasif['source'] = nasif['source'].fillna('')
nasif['source'] = nasif['source'].str.lower()
nasif['source'] = nasif['source'].str.strip()
aniqa['source'] = aniqa['source'].fillna('')
aniqa['source'] = aniqa['source'].str.lower()
aniqa['source'] = aniqa['source'].str.strip()
n = nasif.to_dict('records')
a= aniqa.to_dict('records')

In [49]:
assert len(n) == len(a)

In [50]:
l = []
c=0
for i in range(0, len(n)):
    if n[i]['source'] != a[i]['source']:
        if n[i]['source'] !='n' and a[i]['source'] !='n':
            if not(n[i]['source']=='changelog' and a[i]['source']=='github release note'):
                #print(n[i]['source'],a[i]['source'])
                c+=1
                l.append(i)
s = s.union(set(l))
c

49

In [51]:
fp ={}
l = []
for i in range(0, len(n)):
    t = n[i]['fix_pattern']
    t = t.split(';')
    x = set()
    for e in t:
        if e:
            x.add(e.lower().strip()) 
    t = a[i]['fix_pattern']
    t = t.split(';')
    y = set()
    for e in t:
        if e:
            y.add(e.lower().strip())
    if x!= y:
        l.append(i)
    fp[i]= x.union(y)
s = s.union(set(l))

In [52]:
bp = {}
l = []
for i in range(0, len(n)):
    t = n[i]['breaking_pattern']
    t = t.split(';')
    x = set()
    for e in t:
        if e:
            x.add(e.lower().strip()) 
    t = a[i]['breaking_pattern']
    t = t.split(';')
    y = set()
    for e in t:
        if e:
            y.add(e.lower().strip())
    if x!= y:
        l.append(i)
    bp[i]=x.union(y)
s = s.union(set(l))

In [53]:
len(s)

282

In [54]:
common = []
disagreed = []
for i in range(0,len(n)):
    r = n[i]
    r['row']= i+2
    if i not in s:
        common.append(r)
    else:
        if n[i]['is_doc'] == a[i]['is_doc']:
            r['is_doc'] = n[i]['is_doc']
        else:
            r['is_doc'] =''
        if n[i]['is_fix'] == a[i]['is_fix']:
            r['is_fix'] = n[i]['is_fix']
        else:
            r['is_fix'] =''
        if n[i]['is_uc'] == a[i]['is_uc']:
            r['is_uc'] = n[i]['is_uc']
        else:
            r['is_uc'] =''
        if n[i]['is_br'] == a[i]['is_br']:
            r['is_br'] = n[i]['is_br']
        else:
            r['is_br'] =''
        r['fix_pattern'] = ';'.join(list(fp[i]))
        r['breaking_pattern'] = ';'.join(list(bp[i]))
        disagreed.append(r)


In [55]:
common = pd.DataFrame(common)
disagreed = pd.DataFrame(disagreed)

In [56]:
common.to_csv('common.csv', index=False, header =True)
disagreed.to_csv('disagreed.csv',index=False, header =True)